<div style="display: flex; align-items: center; justify-content: space-between;">
  <div>
    <h3>Modelling Team</h3>
    <ul>
      <li><strong>Dr. Ekaterina Fedotova</strong> - Senior Energy Systems Modeller</li>
      <li><strong>Priyesh Gosai</strong> - Energy Systems Modeler and Training Coordinator</li>
      <li><strong>Albert Chitlango</strong> - Energy Systems Modeler</li>
    </ul>
  </div>
  <div>
    <a href="https://openenergytransition.org/index.html">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="OET">
    </a>
  </div>
</div>


##### 🎯 Learning Objectives  

The aim of this module is to introduce delegates to the environnment that will be used for modelling. We will not cover any contextual aspects of modelliing as this is covered in the main course. 

* Setting up the Google Colab environment
* View the input data used for a PyPSA model. 
* Run the notebook:
   * Import the model. 
   * View input data.
   * Solve network. 
   * View results. 




#### Notebook Setup

In [ ]:
# First-time setup flag
FIRST_RUN = True

if FIRST_RUN:
    # Install latest version of PyPSA with Excel support
    import os
    os.system("pip install pypsa")
    os.system("pip install pypsa[excel]")

    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    
    import shutil
    import subprocess

    FOLDER = 'ich-modeling-2025'
    TARGET_PATH = f'/content/drive/MyDrive/{FOLDER}'
    BACKUP_PATH = f'/content/drive/MyDrive/{FOLDER}_backup'

    # Backup existing folder if it exists
    if os.path.exists(TARGET_PATH):
        print("Backing up existing folder...")
        if os.path.exists(BACKUP_PATH):
            shutil.rmtree(BACKUP_PATH)
        shutil.copytree(TARGET_PATH, BACKUP_PATH)

    # Clone repo if not present
    if not os.path.exists(os.path.join(TARGET_PATH, '.git')):
        subprocess.run(['git', 'clone', 'https://github.com/PriyeshGosai/ich-modeling-2025', TARGET_PATH])
    else:
        # Pull latest updates
        print("Updating existing repo...")
        subprocess.run(['git', '-C', TARGET_PATH, 'pull'])

    os.chdir(TARGET_PATH)
    print(f"Working directory set to: {TARGET_PATH}")


#### Preliminaries

In [ ]:
import pypsa
import pandas as pd
pd.options.plotting.backend = 'plotly' 


INFO:pypsa.io:Imported network mini-grid has buses, carriers, generators, global_constraints, lines, line_types, links, loads, shapes, shunt_impedances, storage_units, transformers, transformer_types


#### Create `network` object

In [ ]:
network = pypsa.Network('mini-grid.xlsx')

#### View static data

In [17]:
network.buses

,v_nom,type,x,y,carrier,unit,location,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,,
Residential Bus,1.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,
Industrial Bus,1.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,
Diesel Bus,1.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,
Solar Bus,1.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,
Wind Bus,1.0,,0.0,0.0,AC,,,1.0,0.0,inf,PQ,,


In [14]:
network.generators

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
Diesel Generator,Diesel Bus,PQ,,40.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,1.0,1.0,1.0,1.0,1.0,0.0
Solar Plant,Solar Bus,PQ,,27.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,1.0,1.0,1.0,1.0,1.0,0.0
Bonus Wind Turbine,Wind Bus,PQ,,150.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,1.0,1.0,1.0,1.0,1.0,0.0
Nordex Wind Turbine,Wind Bus,PQ,,150.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,1.0,1.0,1.0,1.0,1.0,0.0
Loadshedding,Solar Bus,PQ,,1500.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,1.0,1.0,1.0,1.0,1.0,0.0


In [15]:
network.storage_units

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge,cyclic_state_of_charge_per_period,max_hours,efficiency_store,efficiency_dispatch,standing_loss,inflow,p_nom_opt
StorageUnit,,,,,,,,,,,,,,,,,,,,,
Lithium Battery,Solar Bus,PQ,,27.0,0.0,False,0.0,inf,-1.0,1.0,...,False,NaN,False,True,4.814815,0.96,0.960000,0.0,0.0,0.0
Grey Kirk Hydro,Wind Bus,PQ,,5.0,0.0,False,0.0,inf,-1.0,1.0,...,False,NaN,False,True,0.000000,0.00,0.012222,0.0,0.0,0.0
Pumped Hydro,Wind Bus,PQ,,12.0,0.0,False,0.0,inf,-1.0,1.0,...,False,NaN,False,True,4.000000,0.95,0.950000,0.0,0.0,0.0


In [16]:
network.links

,bus0,bus1,type,carrier,efficiency,active,build_year,lifetime,p_nom,p_nom_mod,...,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Link,,,,,,,,,,,,,,,,,,,,,
Alpha Link,Industrial Bus,Residential Bus,,,1.0,True,0,inf,60.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
Beta Link,Diesel Bus,Industrial Bus,,,1.0,True,0,inf,60.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
Gamma Link,Solar Bus,Diesel Bus,,,1.0,True,0,inf,60.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
Delta Link,Wind Bus,Solar Bus,,,1.0,True,0,inf,60.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


#### View timeseries data

In [ ]:
network.generators_t.p_max_pu.plot()

Generator,Solar Plant,Bonus Wind Turbine,Nordex Wind Turbine
snapshot,,,
2024-01-01 00:00:00,0.000,0.336,0.393
2024-01-01 01:00:00,0.000,0.531,0.582
2024-01-01 02:00:00,0.000,0.688,0.726
2024-01-01 03:00:00,0.011,0.653,0.695
2024-01-01 04:00:00,0.086,0.466,0.521
...,...,...,...
2024-12-30 19:00:00,0.000,0.325,0.381
2024-12-30 20:00:00,0.000,0.250,0.303
2024-12-30 21:00:00,0.000,0.195,0.243


In [ ]:
network.storage_units_t.inflow.plot()

StorageUnit,Grey Kirk Hydro
snapshot,
2024-01-01 00:00:00,46.800001
2024-01-01 01:00:00,46.800001
2024-01-01 02:00:00,46.800001
2024-01-01 03:00:00,46.800001
2024-01-01 04:00:00,46.800001
...,...
2024-12-30 19:00:00,2087.999940
2024-12-30 20:00:00,2087.999940
2024-12-30 21:00:00,2087.999940


#### Solve model

In [20]:
network.optimize()

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 6/6 [00:00<00:00, 16.94it/s]
INFO:linopy.io: Writing time: 3.41s


Running HiGHS 1.10.0 (git hash: fd86653): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-wthziydx has 385440 rows; 166440 cols; 586917 nonzeros
Coefficient ranges:
  Matrix [9e-01, 8e+01]
  Cost   [1e+01, 1e+03]
  Bound  [1e+01, 2e+04]
  RHS    [3e-02, 2e+04]
Presolving model
35040 rows, 110413 cols, 171731 nonzeros  0s
26280 rows, 71346 cols, 123904 nonzeros  0s
Dependent equations search running on 26280 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.01s (limit = 1000.00s)
26280 rows, 67488 cols, 120046 nonzeros  1s
Presolve : Reductions: rows 26280(-359160); columns 67488(-98952); elements 120046(-466871)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 1s
      19294    -3.4924596548e-09 Pr: 0(0) 2s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-w

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 166440 primals, 385440 duals
Objective: 0.00e+00
Solver model: available
Solver message: Optimal



Writing the solution to /tmp/linopy-solve-r9bee63c.sol


INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Link-fix-p-lower, Link-fix-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


('ok', 'optimal')

#### View results

In [21]:
network.generators_t.p.plot(title="Generator Power Output")

In [22]:
network.storage_units_t.p.plot(title="Storage Power Output")

---